In [2]:
import pandas as pd

In [3]:
intake24_df = pd.read_csv('files/intake24_survey_file.csv')
print(intake24_df)

                                 Survey ID  User ID                Start time  \
0     5d3e72da-a241-4e8e-837b-021accf9a37c        1  2022-08-03T06:55:02.102Z   
1     5d3e72da-a241-4e8e-837b-021accf9a37c        1  2022-08-03T06:55:02.102Z   
2     5d3e72da-a241-4e8e-837b-021accf9a37c        1  2022-08-03T06:55:02.102Z   
3     5d3e72da-a241-4e8e-837b-021accf9a37c        1  2022-08-03T06:55:02.102Z   
4     5d3e72da-a241-4e8e-837b-021accf9a37c        1  2022-08-03T06:55:02.102Z   
...                                    ...      ...                       ...   
5749  4f7e1c91-c883-438a-b1b5-43bf6f0222cc      263  2022-09-23T11:34:28.178Z   
5750  4f7e1c91-c883-438a-b1b5-43bf6f0222cc      263  2022-09-23T11:34:28.178Z   
5751  4f7e1c91-c883-438a-b1b5-43bf6f0222cc      263  2022-09-23T11:34:28.178Z   
5752  4f7e1c91-c883-438a-b1b5-43bf6f0222cc      263  2022-09-23T11:34:28.178Z   
5753  4f7e1c91-c883-438a-b1b5-43bf6f0222cc      263  2022-09-23T11:34:28.178Z   

               Submission t

# Breakdown (From user to ingredient):

The file has many users.

Each user has many surveys.

Each survey has many meals.

Each meal consists of many food components.

Every food component is marked with a "Nutrition ID code".

In [43]:
def rename_columns(old_name:str, new_name: str, df: pd.DataFrame) -> None:
    df.rename(columns = { old_name : new_name }, inplace=True)
    return None

rename_columns('Survey ID', 'survey_id', intake24_df)
rename_columns('Intake24 food code', 'food_code', intake24_df)
rename_columns('User ID', 'user_id', intake24_df)
rename_columns('Meal ID', 'meal_id', intake24_df)
rename_columns('Nutrient table code', 'heifa_nutrient_id', intake24_df)
rename_columns('Portion size (g/ml)', 'portion_size_consumed', intake24_df)

In [40]:
# Structure is like this:
# - Map user ID to their respective meal and information
# - Map the meal number to the respective ingredients
# - Map the ingredients to their respective nutrient code

user_information = {}

In [66]:
def loop_ingredients(ingredients_df: pd.DataFrame, meal_info_dict: dict) -> dict:

    # What do we need:
    # - The ID: HEIFA
    # - The Intake24 Food Code
    # - The portion consumed
    columns = ['food_code','portion_size_consumed']
    
    food_portion_list = ingredients_df[columns].values.tolist()
    heifa_list = ingredients_df['heifa_nutrient_id'].values.tolist()

    zipped_ingredients = zip(heifa_list, food_portion_list)
    
    for heifa_id, food_portion_list in zipped_ingredients:
        
        food_code = food_portion_list[0]
        portion_size = food_portion_list[1]
        print(f"{heifa_id} for {food_code} of size {portion_size}g.\n\n")

    ...

def loop_meals(meal_ids_list: list, survey_meals_df: pd.DataFrame, survey_meals_dict: dict) -> None:
    
    for meal_id in meal_ids_list:

        survey_meals_dict[meal_id] = {}
        meal_info_dict = survey_meals_dict[meal_id]
        
        ingredients_df = survey_meals_df.query("meal_id == @meal_id")
        loop_ingredients(ingredients_df, meal_info_dict)
        
    ...

In [67]:
# Step 1: Let's split by the survey ID.

# Get the list of survey IDs.
survey_ids_list = intake24_df['survey_id'].unique().tolist()
print(f'Total number of surveys carried out: {len(survey_ids_list)}')

# Loop one survey ID at a time and split the Intake24 dataframe accordingly.
for survey_id in survey_ids_list:

    survey_meals_df = intake24_df.query("survey_id == @survey_id")

    # Add in the user (if it exists)
    user_id = survey_meals_df._get_value(0, 'user_id')
    
    if user_id not in user_information:
        user_information[user_id] = {}

    # Create the empty dictionary for survey information
    # Every meal of the survey will be populated here
    user_information[user_id][survey_id] = {}
    survey_meals_dict = user_information[user_id][survey_id]
    

    # Step 2: Split further the Meal ID
    meal_ids_list = survey_meals_df['meal_id'].unique().tolist()
    loop_meals(meal_ids_list, survey_meals_df, survey_meals_dict)
    break

Total number of surveys carried out: 405


NameError: name 'food_portion_size' is not defined